<a href="https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U02/BinaryClassification_HeartDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="Bilder/ost_logo.png" width="240" height="120" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2022 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

# Binäre Klassifikation mit kontinuierlichen und kategorischen Merkmalen

**Author:** 
- Christoph Würsch, Eastern Switzerland University of Applied Science OST
- [Francois Chollet](https://twitter.com/fchollet)<br>



Diese Übungsserie zeigt, wie eine strukturierte Datenklassifizierung ausgehend von einer rohen
CSV-Datei mit keras vorgenommen werden kann. Die verwendeten Daten enthalten sowohl numerische als auch kategorische Merkmale. Wir verwenden Keras Vorverarbeitungsschichten zur Normalisierung der numerischen Merkmale und zur Vektorisierung (one-hot-coding) der kategorischen Merkmale.

### Der Datensatz

[Unser Datensatz](https://archive.ics.uci.edu/ml/datasets/heart+Disease) wird von der Cleveland Clinic Foundation für Herzkrankheiten zur Verfügung gestellt. Es handelt sich um eine CSV-Datei mit 303 Zeilen. Jede Zeile enthält Informationen über einen Patienten (eine **Stichprobe**), und jede Spalte beschreibt ein Attribut des Patienten (ein **Merkmal**). Wir verwenden die Merkmale, um vorherzusagen, ob ein Patient eine Herzerkrankung hat (**binäre Klassifizierung**).



Hier ist eine Zusammenfassung der Merkmale:

Column| Description| Feature Type
------------|--------------------|----------------------
Age | Age in years | Numerical
Sex | (1 = male; 0 = female) | Categorical
CP | Chest pain type (0, 1, 2, 3, 4) | Categorical
Trestbpd | Resting blood pressure (in mm Hg on admission) | Numerical
Chol | Serum cholesterol in mg/dl | Numerical
FBS | fasting blood sugar in 120 mg/dl (1 = true; 0 = false) | Categorical
RestECG | Resting electrocardiogram results (0, 1, 2) | Categorical
Thalach | Maximum heart rate achieved | Numerical
Exang | Exercise induced angina (1 = yes; 0 = no) | Categorical
Oldpeak | ST depression induced by exercise relative to rest | Numerical
Slope | Slope of the peak exercise ST segment | Numerical
CA | Number of major vessels (0-3) colored by fluoroscopy | Both numerical & categorical
Thal | 3 = normal; 6 = fixed defect; 7 = reversible defect | Categorical
Target | Diagnosis of heart disease (1 = true; 0 = false) | Target

## Setup

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

## (a) Datensatz laden

Wir laden wir die Daten herunter und speichern diese in einen Pandas-Dataframe:

In [ ]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)


In [ ]:
df.to_csv('heart.csv')

Der Datensatz umfasst 303 Proben mit 14 Spalten pro Probe (13 Merkmale, plus die Zielbezeichnung Bezeichnung):

In [ ]:
df.shape

Die letzte Spalte, `target`, gibt an, ob der Patient eine Herzerkrankung hat (`1`) oder nicht (`0`).


## (b) EDA

Hier ist ein kurzer Einblick in die Daten:

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool_)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))


# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, vmin=0.0, center=0, annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});
plt.show()

In [ ]:
features=df.columns[0:-1]
response=df.columns[-1]

features

In [ ]:
for feature in features:
    plt.figure()
    sns.boxplot(data=df,x='target',y=feature)

## (c) Standardisierung und One-hot-Encoding

Die folgenden Merkmale sind kategorische Merkmale, die als ganze Zahlen kodiert sind:

- `Geschlecht`
- `cp` 
- `fbs`
- `restecg`
- `exang`
- `ca`

Wir kodieren diese Merkmale mit **one-hot encoding**. Wir haben zwei Optionen

In [ ]:
print(df.dtypes)

In [ ]:
categorical= ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca', 'thal']

df_onehot=pd.get_dummies(data=df.iloc[:,1:-1],columns=categorical)

features=df_onehot.columns
features


In [ ]:
df_onehot.head()

## (d) Aufteilen in einen Trainings- und Validierungsdatensatz

Wir teilen den Datensatz auf in einen Trainings- und Validierungsdatensatz. Hierfür verwenden wir direkt die Methoden `df.sample` und `df.drop()` eines Pandas-Datenframes `df`. 

In [ ]:
#Generate Dataframe with correct encodiding
numericFeatures=features[0:5]
categoricFeatures=features[5:]

print(numericFeatures)
print(categoricFeatures)

In [ ]:
df_onehot[features].dtypes

In [ ]:
X=df_onehot[features].to_numpy()
y=df['target'].to_numpy()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train

In [ ]:
dg=pd.DataFrame(data=X_train, columns=features)
dg.head()

## (e) Standardisieren der quantitativen, kontinuierlichen Merkmale

In [ ]:
from sklearn.preprocessing import StandardScaler

myScaler=StandardScaler()
Xtrain= np.hstack((myScaler.fit_transform(X_train[:,0:5]),X_train[:,5:]))
Xtest = np.hstack((myScaler.fit_transform(X_test[:,0:5]),X_test[:,5:]))


In [ ]:
np.shape(Xtrain)

In [ ]:
np.shape(y_train.reshape(-1,1))

In [ ]:
trainData=np.hstack((Xtrain,y_train.reshape(-1,1)))
valData =np.hstack((Xtest,y_test.reshape(-1,1)))
fullFeatures=list(features)
fullFeatures.append('target')
print(fullFeatures)


In [ ]:
train_dataframe=pd.DataFrame(data=trainData, columns=fullFeatures)
train_dataframe

In [ ]:
val_dataframe=pd.DataFrame(data=valData, columns=fullFeatures)
val_dataframe

## (f) Generieren eines `tf.data.Datasets`

Die [`tf.data` API](https://www.tensorflow.org/guide/data) ermöglicht es Ihnen, komplexe Eingangs-Pipelines aus einfachen, wiederverwendbaren Teilen aufzubauen. 
- Die Pipeline für ein Bildmodell könnte beispielsweise Daten aus Dateien in einem verteilten Dateisystem aggregieren, zufällige Störungen auf jedes Bild anwenden und zufällig ausgewählte Bilder zum Training zu einem Stapel zusammenführen. 
- Die Pipeline für ein Textmodell kann das Extrahieren von Symbolen aus Rohtextdaten, deren Umwandlung in Einbettungskennungen mit einer Nachschlagetabelle und das Zusammenführen von Sequenzen unterschiedlicher Länge umfassen. 
- Die `tf.data` API ermöglicht es , grosse Datenmengen zu verarbeiten, liest aus verschiedenen Datenformaten und ist in der Lage, komplexe Transformationen durchzuführen.

Die `tf.data`-API führt ein `tf.data.Dataset`-Objekt ein, welches eine Sequenz von Elementen darstellt, wobei jedes Element aus einer oder mehreren Komponenten besteht. In einer Bild-Pipeline könnte ein Element zum Beispiel ein einzelnes Trainingsbeispiel sein, mit je zwei Tensoren, die das Bild und sein Label darstellen.

Es gibt zwei verschiedene Möglichkeiten, ein Dataset zu erstellen:

1. Eine Datenquelle konstruiert ein Dataset aus Daten, die im Speicher oder in einer oder mehreren Dateien gespeichert sind.
2. Eine Datentransformation konstruiert ein Dataset aus einem oder mehreren `tf.data.Dataset`-Objekten.


### Grundlegende Mechanismen

Um eine Eingabe-Pipeline zu erstellen, müssen Sie mit einer Datenquelle beginnen. Um beispielsweise ein Dataset aus Daten im Speicher zu erstellen, können Sie tf.data.Dataset.from_tensors() oder `tf.data.Dataset.from_tensor_slices()` verwenden. Wenn Ihre Eingabedaten in einer Datei im empfohlenen `TFRecord`-Format gespeichert sind, können Sie alternativ `tf.data.TFRecordDataset()` verwenden.

Sobald Sie ein Dataset-Objekt haben, können Sie es in ein neues Dataset umwandeln, indem Sie Methodenaufrufe für das `tf.data.Dataset`-Objekt verketten. Zum Beispiel können Sie Transformationen pro Element wie `Dataset.map()` und Transformationen mit mehreren Elementen wie `Dataset.batch()` anwenden. Eine vollständige Liste der Transformationen finden Sie in der Dokumentation zu `tf.data.Dataset`.

Das Dataset-Objekt ist eine `Python-Iterable`. Dadurch ist es möglich, seine Elemente mit einer `for`-Schleife zu verarbeiten:


In [ ]:
def dataframe_to_dataset(df):
    dg = df.copy()
    labels = dg.pop('target')
    dataset = tf.data.Dataset.from_tensor_slices((dg.values, labels.values))
    dataset = dataset.batch(32).repeat()
    dataset = dataset.shuffle(buffer_size=len(df))
    return dataset

In [ ]:
train_dataset=dataframe_to_dataset(train_dataframe)
val_dataset  =dataframe_to_dataset(val_dataframe)

## (g) Erstellen der Modellarchitektur

In [ ]:
# Returns a placeholder tensor
inputs = layers.Input(shape=(28,))  

# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(32, activation="relu")(inputs)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model1 = keras.Model(inputs, output)

In [ ]:
keras.utils.plot_model(model1, to_file='dropout_classifcation.png', show_shapes=True)


In [ ]:
# The compile step specifies the training configuration.
model1.compile(optimizer='adam',
          loss='binary_crossentropy',
          metrics=['accuracy'])



## (h) Trainieren

In [ ]:
# Train for 50 epochs
history=model1.fit(train_dataset, epochs=50, 
          steps_per_epoch=100,
          validation_data=val_dataset,
          validation_steps=10)

## (i) Lernkurven

In [ ]:
def print_history(history):
    #print(history.history.keys())
    #  "Accuracy"
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.grid(True); plt.show()
    # "Loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.grid(True); plt.show()

In [ ]:
print_history(history)

## (j) Ein etwas anderes Netzwerk

In [ ]:
# Returns a placeholder tensor
inputs = layers.Input(shape=(28,))  

# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(16, activation="relu")(inputs)
x = layers.Dropout(0.3)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)


model2 = keras.Model(inputs, output)

keras.utils.plot_model(model1, to_file='dropout_classifcation.png', show_shapes=True)




In [ ]:
# The compile step specifies the training configuration.
model2.compile(optimizer='adam',
          loss='binary_crossentropy',
          metrics=['accuracy'])

# Trains for 50 epochs
history=model2.fit(train_dataset, epochs=50, 
          steps_per_epoch=100,
          validation_data=val_dataset,
          validation_steps=10)

In [ ]:
plt.figure()
print_history(history)

# A2: Feature Encoding mit `keras`

## (b) Aufteilen in einen Trainings- und Validierungsdatensatz 

In [ ]:
val_dataframe   = df.sample(frac=0.2, random_state=1337)
train_dataframe = df.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

## (c) `tensorflow.data.dataset`-Objekt als Dictionary der einzelnen Merkmale

In [ ]:
def df2dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


In [ ]:
train_ds = df2dataset(train_dataframe)
val_ds = df2dataset(val_dataframe)

In [ ]:
train_ds

## (d) Aufteilen in Batches

In [ ]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

## (e) Preprocessing und Merkmals-Kodierung

In [ ]:
from tensorflow.python.keras.layers.preprocessing.integer_lookup import IntegerLookup
from tensorflow.python.keras.layers.preprocessing.normalization import Normalization
from tensorflow.python.keras.layers.preprocessing.string_lookup import StringLookup


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

## (f) Jedes Merkmal als eigener Input-Tensor

In [ ]:
# Categorical features encoded as integers
sex = keras.Input(shape=(1,), name="sex", dtype="int64")
cp  = keras.Input(shape=(1,), name="cp", dtype="int64")
fbs = keras.Input(shape=(1,), name="fbs", dtype="int64")
restecg = keras.Input(shape=(1,), name="restecg", dtype="int64")
exang   = keras.Input(shape=(1,), name="exang", dtype="int64")
ca = keras.Input(shape=(1,), name="ca", dtype="int64")

# Categorical feature encoded as string
thal = keras.Input(shape=(1,), name="thal", dtype="string")

# Numerical features
age = keras.Input(shape=(1,), name="age")
trestbps = keras.Input(shape=(1,), name="trestbps")
chol    = keras.Input(shape=(1,), name="chol")
thalach = keras.Input(shape=(1,), name="thalach")
oldpeak = keras.Input(shape=(1,), name="oldpeak")
slope   = keras.Input(shape=(1,), name="slope")

all_inputs = [
    sex,
    cp,
    fbs,
    restecg,
    exang,
    ca,
    thal,
    age,
    trestbps,
    chol,
    thalach,
    oldpeak,
    slope,
]


In [ ]:
# Integer categorical features
sex_encoded = encode_categorical_feature(sex, "sex", train_ds, False)
cp_encoded = encode_categorical_feature(cp, "cp", train_ds, False)
fbs_encoded = encode_categorical_feature(fbs, "fbs", train_ds, False)
restecg_encoded = encode_categorical_feature(restecg, "restecg", train_ds, False)
exang_encoded = encode_categorical_feature(exang, "exang", train_ds, False)
ca_encoded = encode_categorical_feature(ca, "ca", train_ds, False)

# String categorical features
thal_encoded = encode_categorical_feature(thal, "thal", train_ds, True)

# Numerical features
age_encoded = encode_numerical_feature(age, "age", train_ds)
trestbps_encoded = encode_numerical_feature(trestbps, "trestbps", train_ds)
chol_encoded = encode_numerical_feature(chol, "chol", train_ds)
thalach_encoded = encode_numerical_feature(thalach, "thalach", train_ds)
oldpeak_encoded = encode_numerical_feature(oldpeak, "oldpeak", train_ds)
slope_encoded = encode_numerical_feature(slope, "slope", train_ds)

all_features = layers.concatenate(
    [
        sex_encoded,
        cp_encoded,
        fbs_encoded,
        restecg_encoded,
        exang_encoded,
        slope_encoded,
        ca_encoded,
        thal_encoded,
        age_encoded,
        trestbps_encoded,
        chol_encoded,
        thalach_encoded,
        oldpeak_encoded,
    ]
)

## (g) Netzarchitektur und Training

In [ ]:
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

## (h) Darstellung

In [ ]:
# `rankdir='LR'` is to make the graph horizontal.
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

## (h) Inferenz auf neue Daten

Um eine Vorhersage für eine neue Probe zu erhalten, können Sie einfach `model.predict()` aufrufen. Es gibt nur zwei Dinge, die Sie tun müssen:
1. Skalare in eine Liste einpacken, um eine Stapeldimension zu haben (Modelle verarbeiten nur Datenstapel, nicht einzelne Stichproben)
2. Rufen Sie `convert_to_tensor` für jedes Merkmal auf

In [ ]:
sample = {
    "age": 60,
    "sex": 1,
    "cp": 1,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 2,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 3,
    "ca": 0,
    "thal": "fixed",
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(input_dict)

print(
    "This particular patient had a %.1f percent probability "
    "of having a heart disease, as evaluated by our model." % (100 * predictions[0][0],)
)